# Machine Learning

## Set up

In [3]:
!pip install lightgbm

In [1]:
import lighgbm as lgb

ModuleNotFoundError: No module named 'lighgbm'

In [1]:
# Set up folders
from EDA_functions import folders_set_up
import os

# Work with datarames
import pandas as pd
import numpy as np

# Charts
import seaborn as sns
from matplotlib import pyplot as plt

# X, Y preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Light GBM
import lightgbm as lgb

# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree

# Neural Network
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout
from keras.optimizers import Adam, SGD

# Pipeline
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'lightgbm'

### Folders
Run the code below if you have the following structure:
- Group-project: GitHub folder
- 01 Input
- 02 Output

In [4]:
analysis_folder, input_folder, output_folder = folders_set_up.generate_folders()

## Import and merge data

### Datasets

In [5]:
#Title-level dataset
titles_df = pd.read_pickle(
    os.path.join(output_folder, 'English_fiction_pre_PCA_3_with_embeddings')
)

titles_df.columns

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'infoLink', 'categories', 'reviews number', 'average rating',
       'median rating', 'min review date', 'max review date',
       'weighted rating', 'date', 'year', 'description_language', 'Embedding'],
      dtype='object')

In [6]:
# indices are missing in the file above, we get them from another dataset
index_df = pd.read_csv(
    os.path.join(output_folder, 'English_fiction_pre_PCA_3.csv')
)

index_df = index_df[['Title', 'index']]

In [7]:
# NLP
# descriptions_df

### Merge

In [8]:
# Merge titles dataframe with indices dataframe

df = pd.merge(
    titles_df,
    index_df,
    on = 'Title',
    how = 'left'
)

df.columns

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'infoLink', 'categories', 'reviews number', 'average rating',
       'median rating', 'min review date', 'max review date',
       'weighted rating', 'date', 'year', 'description_language', 'Embedding',
       'index'],
      dtype='object')

In [25]:
# Merge titles dataframe and description PCA

# df = pd.merge(
#     titles_df,
#     descriptions_df,
#     on = 'index',
#     how = 'left'
# )

### Format data

In [9]:
df.dtypes

Title                    object
description              object
authors                  object
image                    object
previewLink              object
publisher                object
infoLink                 object
categories               object
reviews number            int64
average rating          float64
median rating           float64
min review date          object
max review date          object
weighted rating         float64
date                     object
year                    float64
description_language     object
Embedding                object
index                     int64
dtype: object

#### Date

In [15]:
dates_columns = ['min review date', 'max review date', 'date']

for date in dates_columns:
    # get date from strings with time
    df[date] = df[date].str.split().str[0]
    # convert in datetime
    df[date] = pd.to_datetime(df[date])

min review date
max review date
date


In [17]:
df[dates_columns].isna().sum()

min review date    0
max review date    0
date               0
dtype: int64

### Clean data
Most of the cleaning is done in '02 Consolidate books dataset':
- English description
- category containing the word 'fiction'
- non-missing date
- non-missing author
- non-missing publisher
- non-missing cover image

## X and y set up

### Train test split

In [26]:
# NOTE: we work on a subset of data for now to make the ML run faster
df = df.sample(n=1000, random_state=42)

In [28]:
# Create X and y including all X features and all all teh possible target variables
# NOTE: we will have to add the description PCA in X_features
X_features = ['authors', 'publisher', 'date']
X = df[X_features+['Embedding']]
y = df[['average rating', 'weighted rating']]

In [29]:
# Create train test split

# Need to create train test split for different combinations of data
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size= 0.2, 
    random_state= 42,
)

In [30]:
# We are going to run two models for two target variables
# - Target variable: Average rating
#   - baseline (i.e. excluding image embeddings)
#   - including image embeddings
# - Target variable: weighted rating
#   - baseline (i.e. excluding image embeddings)
#   - including image embeddings

# We therefore need to create the following datsets
# - X train and X test with embeddings
# - X train and X text without embeddings
# - y train and y test using average rating
# - y train and y test using weighted rating

X_baseline_train = X_train.drop('Embedding', axis = 1)
X_baseline_test = X_test.drop('Embedding', axis = 1)
X_images_train = X_train
X_images_test = X_test

y_avg_r_train = y_train['average rating']
y_avg_r_test = y_test['average rating']

y_wr_train = y_train['weighted rating']
y_wr_test = y_test['weighted rating']

### Scale variables

No need to scale variables for now

## Lightgbm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 8.6 MB/s eta 0:00:01
  distutils: /private/var/folders/1z/cxkb68zj6_s08n7300y_mzm00000gn/T/pip-build-env-dywf0zex/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/1z/cxkb68zj6_s08n7300y_mzm00000gn/T/pip-build-env-dywf0zex/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/1z/cxkb68zj6_s08n7300y_mzm00000gn/T/pip-build-env-dywf0zex/normal'
  distutils: /private/var/folders/1z/cxkb68zj6_s08n7300y_mzm00000gn/T/pip-build-env-dywf0zex/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/1z/cxkb68zj6_s08n7300y_mzm00000gn/T/pip-build-env-dywf0zex/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

## Random forest

-> Questions/notes:
- any hyperparameter we need to think about?
- which metric are we using to evaluate the model?

### Set up

In [31]:
# Set up RF
rf = RandomForestRegressor()

### Train model

In [32]:
# Fit data
rf.fit(X_baseline_train, y_avg_r_test)

ValueError: could not convert string to float: "['Giovanna Fletcher']"

### Predict

In [ ]:
# Make predictions
rf_pred = rf.predict(X_test)

### Feature importance

In [ ]:
# Display the feature importance.

# get importance of features and assign names
rf_importances = rf.feature_importances_
rf_importances = pd.DataFrame({'feature':X.columns, 'importance': rf_importances})
# sort dataset by importance
rf_importances = rf_importances.sort_values(by = 'importance', ascending = False)

# Draw chart
ax = sns.barplot(
    data = rf_importances,
    x = 'importance',
    y = 'feature'
)

for index, value in enumerate(rf_importances['importance']):
    ax.text(value, index, f'{value:.4f}', ha='left', va='center', fontsize=10)

### Decision tree of the forest

In [4]:
#Using a decision tree with a max_leaf_nodes of 3, plot the decision tree

dt_max_3 = DecisionTreeRegressor(max_depth=3)
dt_max_3.fit(X_train,y_train)
dt_max_3_predictions = dt_max_3.predict(X_test)

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=300)
tree.plot_tree(
    dt_max_3,
    max_depth = 3, 
    feature_names=X_train.columns,  
    class_names=True,
    filled=True,
    fontsize=8
)
plt.show()


### Evaluate model

## Neural Network

-> Questions/notes:
Inputs to choose:
- number of layers
- activation functions
- Use softmax in the last layer to obtain the probability distribution of the outcome?
- optimizer: Adam? sdg?
- loss function
- add dense layers to avoid overfitting?
- number of epochs
- which metric to use to evaluate the model?

- Use gridsearch to optimise hyperparameters?

### Set up

In [ ]:
# get number of inputs - second element of shape (i.e. number of columns in X)
input_shape = X.shape[1]

# neurons number
n_neurons = 512

# define a model
model = keras.Sequential()

# Add input layer
model.add(layers.Dense(
            n_neurons, # number of neurons
            input_dim = input_shape, # number of inputs 
            activation = 'tanh' # activation faunction
            ))

# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

# To change activation function: Output: only want one neuron in the last layer = no activation because we want an output that is a continuous variable (same as saying activation = linear)
model.add(layers.Dense(1, activation = 'linear'))

model.summary()

### Compile

In [ ]:
# Compile model

# Concrete and wine datasets (output is continuous variable)
model.compile(
    optimizer='sgd', 
    loss='mean_squared_error', 
    metrics=['mae'])

# Shallow Network (numbers recognition - output is categorical var)
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.01),  # lr = learning rate
    metrics=['accuracy']
)

# Auto purchase dataset (y is continuous)
model.compile(
    optimizer='adam',
    loss='mean_squared_error')

### Tune model?

In [ ]:
# Grid search?

### Train model

-> Note:
- this will have to be done including and excluding images

In [ ]:
# Train model
epochs_hist = model.fit(
    X_train, # input
    y_train, # output
    epochs=100, # number of iterations
    batch_size=50, # number of observations taken to train the data - 1030 obs/50 -> there are 17 groups (observations are taken once for epoch) so model is trained 17 times in each epoch
    verbose=1,
    validation_data = (X_test, y_test),
    shuffle = True
    #validation_split=0.2,    
)

### Predict

In [ ]:
# Predict
y_pred = model.predict(X_test)

### Evaluate model

In [ ]:
# evaluate the model (it will give the metric specified when model is compiled)
score = model.evaluate(
    X_test,
    y_test,
    verbose=1
)

In [ ]:
# Visualise NN

# Plotting Loss And Root Mean Square Error For both Training And Test Sets
plt.plot(epochs_hist.history['mae'])
plt.plot(epochs_hist.history['val_mae'])
plt.title('MAE')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(epochs_hist.history['loss'])
plt.plot(epochs_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('4.png')
plt.show()

In [ ]:
# Interpretation of model performance - example with y continuous
test_predictions_ = model.predict(test_df).flatten()
test_labels_ = test_labels.to_numpy().flatten()

_, ax = plt.subplots(figsize=(14,8))
plt.scatter(
    test_labels_,
    test_predictions_,
    alpha=0.6,
    color='#ff7043',
    lw=1,
    ec='black'
)

lims = [
    0,
    max(test_predictions_.max(), test_labels_.max())
]

plt.plot(lims, lims, lw=1, color='#00acc1')
plt.tight_layout()
plt.show()

## Set up pipeline

-> Questions/notes:
- how to integrate NN in the pipeline?

In [ ]:
# Example
# Define pipeline steps
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Feature scaling
    ('rf', rf)  # Random Forest classifier
])

nn_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Feature scaling
    ('nn', model)  # Neural Network classifier
])

# Fit Random Forest pipeline
rf_pipeline.fit(X_train, y_train)

# Fit Neural Network pipeline
nn_pipeline.fit(X_train, y_train)

# Evaluate models
rf_accuracy = rf_pipeline.score(X_test, y_test)
nn_accuracy = nn_pipeline.score(X_test, y_test)

print("Random Forest Accuracy:", rf_accuracy)
print("Neural Network Accuracy:", nn_accuracy)
